# Generate F plasmid for GSMM

### Load packages

In [1]:
import cobra
from cobra import Model, Reaction, Metabolite
import numpy as np 
import pandas as pd
import re
import string
import warnings
warnings.filterwarnings('ignore')
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model 

### Make metabolite dictionaries

In [2]:
metDict = {
    'atp': 'datp_c',        # ATP,              ChEBI 15422
    'ctp': 'dctp_c',        # CTP,              ChEBI 17677
    'gtp': 'dgtp_c',        # GTP,              ChEBI 15996
    'ttp': 'dttp_c',        # UTP,              ChEBI 15713
    'A': 'ala__L_c',        # Alaline,          ChEBI 16977
    'R': 'arg__L_c',        # Arginine,         ChEBI 16467
    'N': 'asn__L_c',        # Asparagine,       ChEBI 17196
    'D': 'asp__L_c',        # Aspartate,        ChEBI 17053
    'C': 'cys__L_c',        # Cysteine,         ChEBI 17561
    'Q': 'gln__L_c',        # Glutamine,        ChEBI 18050
    'E': 'glu__L_c',        # Glutamate,        ChEBI 16015
    'G': 'gly_c',          # Glycine,          ChEBI 15428
    'H': 'his__L_c',        # Histidine,        ChEBI 15971
    'I': 'ile__L_c',        # Isoleucine,       ChEBI 17191
    'L': 'leu__L_c',        # Leucine,          ChEBI 15603
    'K': 'lys__L_c',        # Lysine,           ChEBI 18019
    'M': 'met__L_c',        # Methionine,       ChEBI 16643
    'F': 'phe__L_c',        # Phenylalanine,    ChEBI 17295
    'P': 'pro__L_c',        # Proline,          ChEBI 17203
    'S': 'ser__L_c',        # Serine,           ChEBI 17115
    'T': 'thr__L_c',        # Threonine,        ChEBI 16857
    'W': 'trp__L_c',        # Tryptophan,       ChEBI 16828
    'Y': 'tyr__L_c',        # Tyrosine,         ChEBI 17895
    'V': 'val__L_c',        # Valine,           ChEBI 16414
    'h2o': 'h2o_c',        # H2O
    'adp': 'adp_c',        # ADP
    'Pi': 'pi_c',          # Phosphate
    'h': 'h_c',            # Hydrogen [Proton]
    'PPi': 'ppi_c',        # Pyrophosphate
}
# Nucleotide dictionary with molecular weights
# Source: ChEBI https://www.ebi.ac.uk/chebi/
ntpsDict = {
    'atp': 507.181,     # ATP,              ChEBI 15422
    'gtp': 483.15644,   # GTP,              ChEBI 17677
    'ctp': 523.18062,   # CTP,              ChEBI 15996
    'ttp': 498.16770,   # TTP,              ChEBI 15713
}
# Amino Acids dictionary with molecular weights
# Source: ChEBI https://www.ebi.ac.uk/chebi/
aaDict = {
    'A': 89.09322,      # Alanine,          ChEBI 16977
    'R': 174.201,       # Arginine,         ChEBI 16467
    'N': 132.118,       # Asparagine,       ChEBI 17196
    'D': 133.1027,      # Aspartate,        ChEBI 17053
    'C': 121.158,       # Cysteine,         ChEBI 17561
    'Q': 146.14458,     # Glutamine,        ChEBI 18050
    'E': 147.1293,      # Glutamate,        ChEBI 16015
    'G': 75.06664,      # Glycine,          ChEBI 15428
    'H': 155.15468,     # Histidine,        ChEBI 15971
    'I': 131.17296,     # Isoleucine,       ChEBI 17191
    'L': 131.17296,     # Leucine,          ChEBI 15603
    'K': 146.18764,     # Lysine,           ChEBI 18019
    'M': 149.21238,     # Methionine,       ChEBI 16643
    'F': 165.18918,     # Phenylalanine,    ChEBI 17295
    'P': 115.1305,      # Proline,          ChEBI 17203
    'S': 105.09262,     # Serine,           ChEBI 17115
    'T': 119.1192,      # Threonine,        ChEBI 16857
    'W': 204.22526,     # Tryptophan,       ChEBI 16828
    'Y': 181.18858,     # Tyrosine,         ChEBI 17895
    'V': 117.14638,     # Valine,           ChEBI 16414
}
# Misc. dictionary with molecular weights
# Source: ChEBI https://www.ebi.ac.uk/chebi/
miscDict = {
    'PPi': 173.94332,   # Pyrophosphate,    ChEBI 18361
}
# Nucleotides List
ntpsMets    = list(ntpsDict.keys())
# Amino Acids List
aaMets      = list(aaDict.keys())
# Avogadro's Number
N_A         = 6.0221409e+23
# Energy requirement coefficients
# Source: Haynie, D. (2008). Biological Thermodynamics (2nd ed.). Cambridge: Cambridge University Press.
k_atp       = 4
k_ppi       = 1

datp_c = Metabolite(metDict['atp'])
dctp_c = Metabolite(metDict['ctp'])
dgtp_c = Metabolite(metDict['gtp'])
dttp_c = Metabolite(metDict['ttp'])
ala__L_c = Metabolite(metDict['A'])
arg__L_c = Metabolite(metDict['R'])
asn__L_c = Metabolite(metDict['N'])
asp__L_c = Metabolite(metDict['D'])
cys__L_c = Metabolite(metDict['C'])
gln__L_c = Metabolite(metDict['Q'])
glu__L_c = Metabolite(metDict['E'])
gly_c = Metabolite(metDict['G'])
his__L_c = Metabolite(metDict['H'])
ile__L_c = Metabolite(metDict['I'])
leu__L_c = Metabolite(metDict['L'])
lys__L_c = Metabolite(metDict['K'])
met__L_c = Metabolite(metDict['M'])
phe__L_c = Metabolite(metDict['F'])
pro__L_c = Metabolite(metDict['P'])
ser__L_c = Metabolite(metDict['S'])
thr__L_c = Metabolite(metDict['T'])
trp__L_c = Metabolite(metDict['W'])
tyr__L_c = Metabolite(metDict['Y'])
val__L_c = Metabolite(metDict['V'])
adp_c = Metabolite(metDict['adp'])
h2o_c = Metabolite(metDict['h2o'])
h_c   = Metabolite(metDict['h'])
pi_c  = Metabolite(metDict['Pi'])
ppi_c = Metabolite(metDict['PPi'])

### Make pilus gene dictionary 

In [3]:
#based on Braggagnolo et al 2020
max_pilin_units = 1e6
single_pilus_dict = {
    'FinO' : max_pilin_units * 0.0001, #transcriptional regulation
    'TraJ' : max_pilin_units * 0.0001, #transcriptional regulation
    'TraR' : max_pilin_units * 0.0001, #transcriptional regulation
    'TraA' : max_pilin_units, #propilin component, 12.8 units per helical turn in a 5 start helix, 
    #65 units per 13.2 Angstroms of length, maximum pilus length of 20microns leads to 1e6 max per pilus 
    #from Braggagnolo et al 2020
    'TraQ' : max_pilin_units * 0.25, #transient attachment to traA so can be reused
    'TraX' : max_pilin_units * 0.5, #acetylates propilin
    'TraB' : max_pilin_units * 0.001, #t4ss core protein
    'TraK' : max_pilin_units * 0.001, #t4ss core protein
    'TraV' : max_pilin_units * 0.001, #t4ss core protein
    'TraP' : max_pilin_units * 0.01, #assembly/extension
    'TraE' : max_pilin_units * 0.01, #assembly/extension
    'TraL' : max_pilin_units * 0.01, #assembly/extension
    'TraC' : max_pilin_units * 0.01, #assembly/extension
    'TraW' : max_pilin_units * 0.01, #assembly/extension
    'TrbC' : max_pilin_units * 0.01, #assembly/extension
    'TraF' : max_pilin_units * 0.01, #assembly/extension
    'TrbB' : max_pilin_units * 0.01, #assembly/extension
    'TraH' : max_pilin_units * 0.01, #retraction
    'TrbI' : max_pilin_units * 0.01, #retraction
    'TraN' : max_pilin_units * 0.001, #mating pair stabilization
    'TraG' : max_pilin_units * 0.001, #mating pair stabilization
    'TraU' : max_pilin_units * 0.001, #mating pair stabilization
    'TraI' : max_pilin_units * 0.001, #relaxase
    'TraY' : max_pilin_units * 0.001, #relaxase
    'TraM' : max_pilin_units * 0.001, #relaxase
    'TraD' : max_pilin_units * 0.001, #relaxase
    'TraT' : max_pilin_units * 0.0001, #surface exclusion
    'TraS' : max_pilin_units * 0.0001, #entry exclusion
    'TrbD' : max_pilin_units * 0.00001, #unknown function
    'TrbE' : max_pilin_units * 0.00001, #unknown function
    'TrbF' : max_pilin_units * 0.00001, #unknown function
    'TrbA' : max_pilin_units * 0.00001, #unknown function
    'TrbG' : max_pilin_units * 0.00001, #unknown function
    'TrbJ' : max_pilin_units * 0.00001 #unknown function
}

### Make M13 gene dictionary

In [6]:
#based on Wang et al
phage_dict = {
    'III' : 5, #minor capsid protein
    'VI' : 5, #minor capsid protein
    'VII' : 5, #minor capsid protein
    'IX' : 5, #minor capsid protein
    'VIII' : 2700, #major capsin protein
    'II' : 10, #initiates DNA replication, koskoe
    'X' : 10, #inhibits replication
    'V' : 1500, #packaging protein, encapsulates DNA
    'I' : 15, #part of gated channel
    'IV' : 15 #conners et al, gated channel 
}

### Define the plasmid and phage generating functions

In [53]:
def generate_model_with_plasmid(number_of_pili, model):
    #Import the F plasmid and Tn10 sequences from Genbank
    plasmid = "../models/genbank/f-plasmid.txt"
    with open(plasmid,newline = '') as pf:
        plasmidFile = pf.readlines()
    plasmidData = str(plasmidFile)

    tn10 = "../models/genbank/tn10.txt"
    with open(tn10,newline = '') as tf:
        tn10File = tf.readlines()
    tn10Data = str(tn10File)
    
    #Calculate the nucleotide code of the F plasmid and Tn10 
    startG  = [jj for jj, s in enumerate(plasmidFile) if 'ORIGIN' in s]
    endG    = [jj for jj, s in enumerate(plasmidFile) if '//' in s]
    startG  = int(''.join(map(str,startG)))
    endG    = int(''.join(map(str,endG)))
    startG  = startG + 1

    startG_t  = [jj for jj, s in enumerate(tn10File) if 'ORIGIN' in s]
    endG_t    = [jj for jj, s in enumerate(tn10File) if '//' in s]
    startG_t  = int(''.join(map(str,startG_t)))
    endG_t    = int(''.join(map(str,endG_t)))
    startG_t  = startG_t + 1
    
    regex           = re.compile('[^a-zA-Z]')
    plasmidGenome     = str(''.join(plasmidFile[startG:endG]))
    plasmidGenome     = regex.sub('',plasmidGenome)

    tn10Genome     = str(''.join(tn10File[startG_t:endG_t]))
    tn10Genome     = regex.sub('',tn10Genome)
    
    Cg = 1 #genome copy number, for F plasmid usually only 1 and never more than 2
    countA  = plasmidGenome.count('a')
    countC  = plasmidGenome.count('c')
    countG  = plasmidGenome.count('g')
    countT  = plasmidGenome.count('t')   
    antiA   = countT
    antiC   = countG
    antiG   = countC
    antiT   = countA

    countA_t  = tn10Genome.count('a')
    countC_t   = tn10Genome.count('c')
    countG_t   = tn10Genome.count('g')
    countT_t   = tn10Genome.count('t')   
    antiA_t    = countT_t 
    antiC_t    = countG_t 
    antiG_t    = countC_t 
    antiT_t    = countA_t 

    totNTPS   = (Cg * (countA + countC + countG + countT + antiA + antiC + antiG + antiT + countA_t + countC_t + countG_t + countT_t + antiA_t + antiC_t + antiG_t + antiT_t))

    # mol.ntps/mol.plasmid
    V_a = (Cg*(countA + antiA + countA_t + antiA_t))
    V_c = (Cg*(countC + antiC + countC_t + antiC_t))
    V_g = (Cg*(countG + antiG + countG_t + antiG_t))
    V_t = (Cg*(countT + antiT + countT_t + antiT_t))
    # g.ntps/mol.plasmid
    G_a = V_a * ntpsDict["atp"]
    G_c = V_c * ntpsDict["ctp"]
    G_g = V_g * ntpsDict["gtp"]
    G_t = V_t * ntpsDict["ttp"]
    
    # Calculate the protein cost of the pilus-producing regions of the F plasmid
    pilus_genes = list(single_pilus_dict.keys())
    pilus_genes.remove('FinO')
    pilus_genes.remove('TraX')
    pilus_genes.remove('TraR')
    pilus_gene_IDs = dict.fromkeys(pilus_genes)

    for g in range(0,len(pilus_genes)):
        proteinString = '/product="conjugal transfer protein ' + pilus_genes[g] + '"'
        tempLine = [jj for jj, s in enumerate(plasmidFile) if proteinString in s]
        val = tempLine[0]
        pilus_gene_IDs[pilus_genes[g]] = val

    pilus_gene_IDs["TraX"] = [jj for jj, s in enumerate(plasmidFile) if "WP_000205715.1" in s][0]
    
    linesOfInterest = list(pilus_gene_IDs.values())
    linesOfInterest.sort()
    conjugalProteins = plasmidFile[linesOfInterest[0]:linesOfInterest[-1] + 25]

    translationLines = [jj for jj, s in enumerate(conjugalProteins) if "/translation" in s]
    geneLines = [jj for jj, s in enumerate(conjugalProteins) if "gene            " in s] #number of tab spaces matters!
    productLines = [jj for jj, s in enumerate(conjugalProteins) if '/product="conjugal' in s]

    translationDictionary = {}
    for i in range(0, len(productLines) - 1):
        start = productLines[i]
        end = productLines[i + 1]
        temp = conjugalProteins[start:end]
        translationStart = [jj for jj, s in enumerate(temp) if "/translation" in s][0]
        translationEnd = [jj for jj, s in enumerate(temp) if "gene            " in s][0]
        productString = [jj for jj, s in enumerate(temp) if '/product="conjugal transfer protein' in s][0]
        geneString = temp[productString].split()[-1]
        geneString = geneString.translate(str.maketrans('', '', string.punctuation))
        translationRegion = temp[translationStart:translationEnd]
        npReg = re.compile('/translation=')
        tempgene  = str(translationRegion)
        tempgene  = npReg.sub('',tempgene)
        tempgene  = regex.sub('',tempgene)
        translationDictionary[geneString] = tempgene

    endDict = 31
    start = productLines[endDict]
    end = productLines[endDict] + 10
    temp = conjugalProteins[start:end]
    translationStart = [jj for jj, s in enumerate(temp) if "/translation" in s][0]
    translationEnd = [jj for jj, s in enumerate(temp) if "gene            " in s][0]
    geneString = "TraX"
    translationRegion = temp[translationStart:translationEnd]
    npReg = re.compile('/translation=')
    tempgene  = str(translationRegion)
    tempgene  = npReg.sub('',tempgene)
    tempgene  = regex.sub('',tempgene)
    translationDictionary[geneString] = tempgene
    
    pilus_proteins = list(translationDictionary.keys())
    np_list = np.zeros((20,1))
    for xx in pilus_proteins:
        aaCount = np.zeros((20,1))
        protein = translationDictionary[xx]
        for ii in range(len(aaMets)):
            aaCount[ii,0] = protein.count(aaMets[ii])
        finalarray = aaCount * single_pilus_dict[xx]
        numpy_adjusted = np.array(finalarray)
        np_list = np_list + finalarray

    pilus_protein_content = np_list * number_of_pili
    
    # Calculate the protein cost of the non pilus-producing regions of the F plasmid
    singleProductLines = [jj for jj, s in enumerate(plasmidFile) if '/product=' in s]
    singleTranslationLines = [jj for jj, s in enumerate(plasmidFile) if "/translation" in s]
    singleGeneLines = [jj for jj, s in enumerate(plasmidFile) if "gene            " in s] #number of tab spaces matters!

    fullTranslationString = ''
    for i in range(0, len(singleProductLines) - 1):
        start = singleProductLines[i]
        end = singleProductLines[i + 1]
        temp = plasmidFile[start:end]
        if not [jj for jj, s in enumerate(temp) if "/translation" in s]:
            continue
        else:
            translationStart = [jj for jj, s in enumerate(temp) if "/translation" in s][0]
            translationEnd = [jj for jj, s in enumerate(temp) if "gene            " in s][0]
            translationRegion = temp[translationStart:translationEnd]
            npReg = re.compile('/translation=')
            tempgene  = str(translationRegion)
            tempgene  = npReg.sub('',tempgene)
            tempgene  = regex.sub('',tempgene)
            fullTranslationString = fullTranslationString + tempgene
    
    endDict = 241
    start = singleProductLines[endDict]
    end = singleProductLines[endDict] + 10
    temp = plasmidFile[start:end]
    translationStart = [jj for jj, s in enumerate(temp) if "/translation" in s][0]
    translationEnd = [jj for jj, s in enumerate(temp) if "ORIGIN" in s][0]
    translationRegion = temp[translationStart:translationEnd]
    npReg = re.compile('/translation=')
    tempgene  = str(translationRegion)
    tempgene  = npReg.sub('',tempgene)
    tempgene  = regex.sub('',tempgene)
    fullTranslationString = fullTranslationString + tempgene


    nonPilusCount = np.zeros((20,1))
    for ii in range(len(aaMets)):
        nonPilusCount[ii,0] = fullTranslationString.count(aaMets[ii])

    tn10ProductLines = [jj for jj, s in enumerate(tn10File) if '/product=' in s]
    tn10TranslationLines = [jj for jj, s in enumerate(tn10File) if "/translation" in s]
    tn10GeneLines = [jj for jj, s in enumerate(tn10File) if "gene            " in s] #number of tab spaces matters!

    fulltn10TranslationString = ''
    for i in range(0, len(tn10ProductLines) - 1):
        start = tn10ProductLines[i]
        end = tn10ProductLines[i + 1]
        temp = tn10File[start:end]
        if not [jj for jj, s in enumerate(temp) if "/translation" in s]:
            continue
        else:
            translationStart = [jj for jj, s in enumerate(temp) if "/translation" in s][0]
            translationEnd = [jj for jj, s in enumerate(temp) if "gene            " in s][0]
            translationRegion = temp[translationStart:translationEnd]
            npReg = re.compile('/translation=')
            tempgene  = str(translationRegion)
            tempgene  = npReg.sub('',tempgene)
            tempgene  = regex.sub('',tempgene)
            fulltn10TranslationString = fulltn10TranslationString + tempgene
            
    endDict = 4
    start = tn10ProductLines[endDict]
    end = tn10ProductLines[endDict] + 35
    temp = tn10File[start:end]
    translationStart = [jj for jj, s in enumerate(temp) if "/translation" in s][0]
    translationEnd = [jj for jj, s in enumerate(temp) if "mobile_element" in s][0]
    translationRegion = temp[translationStart:translationEnd]
    npReg = re.compile('/translation=')
    tempgene  = str(translationRegion)
    tempgene  = npReg.sub('',tempgene)
    tempgene  = regex.sub('',tempgene)
    fulltn10TranslationString = fulltn10TranslationString + tempgene

    tn10Count = np.zeros((20,1))
    for ii in range(len(aaMets)):
        tn10Count[ii,0] = fulltn10TranslationString.count(aaMets[ii])
    
    # Calculate total amino acid cost
    totAA = tn10Count + pilus_protein_content + nonPilusCount
    
    # mol.aa/mol.plasmid
    V_aa    = np.zeros((20,1))
    for ii in range(len(aaMets)):
        V_aa[ii,0] = totAA[ii]
    # g.a/mol.plasmid
    G_aa    = np.zeros((20,1))
    for ii in range(len(aaMets)):
        G_aa[ii,0] = V_aa[ii] * aaDict[aaMets[ii]]
        
    # Combine the final counts for nucleotide and protein costs
    
    M_v = (G_a + G_c + G_g + G_t) + G_aa.sum()
    
    # Nucleotides [mmol.ntps/g.virus]
    S_atp = 1000 * (V_a/M_v)
    S_ctp = 1000 * (V_c/M_v)
    S_gtp = 1000 * (V_g/M_v)
    S_ttp = 1000 * (V_t/M_v)

    # Amino acids [mmol.aa/g.virus]
    S_aa    = np.zeros((20,1))
    for ii in range(len(aaMets)):
        S_aa[ii] = 1000 * (V_aa[ii]/M_v)

    # Genome: Phosphodiester bond formation products [Pyrophosphate]
    genTemp = (((countA + countC + countG + countT + countA_t + countC_t + countG_t + countT_t) * k_ppi) - k_ppi)
    genRep  = (((antiA + antiC + antiG + antiT + antiA_t + antiC_t + antiG_t + antiT_t) * k_ppi) - k_ppi)
    genTot  = genTemp + genRep
    V_ppi   = genTot
    S_ppi   = 1000 * (V_ppi/M_v)

    # Protome: Peptide bond formation [ATP + H2O]
    # Note: ATP used in this process is denoated as ATPe/Ae [e = energy version]
    ppAe    = ((pilus_protein_content.sum() * k_atp) - k_atp)
    tnAe    = ((tn10Count.sum() * k_atp) - k_atp)
    npAe    = ((nonPilusCount.sum() * k_atp) - k_atp)
    ppTot   = ppAe + tnAe + npAe
    V_Ae    = ppTot
    S_Ae    = 1000 * (V_Ae/M_v)
    
    # Generate reaction
    
    # Left-hand terms: Nucleotides
    # Note: ATP term is a summation of genome and energy requirements
    S_ATP   = (S_atp + S_Ae) * -1
    S_CTP   = S_ctp * -1
    S_GTP   = S_gtp * -1
    S_TTP   = S_ttp * -1
    # Left-hand terms: Amino Acids
    S_AA    = S_aa * -1
    S_AAf   = dict()
    for ii in range(len(aaMets)):
        S_AAf[aaMets[ii]] = S_AA[ii,0]
    # Left-hand terms: Energy Requirements
    S_H2O   = S_Ae * -1
    # Right-hand terms: Energy Requirements
    S_ADP   = S_Ae
    S_Pi    = S_Ae
    S_H     = S_Ae
    S_PPi   = S_ppi
    
    Name = "plasmid"
    reaction_name       = Name + '_F'
    plasmid_reaction      = Reaction(reaction_name)
    plasmid_reaction.lower_bound              = 0.0
    plasmid_reaction.upper_bound              = 1000.0
    plasmid_reaction.add_metabolites(({
        datp_c: S_ATP,
        dctp_c: S_CTP,
        dgtp_c: S_GTP,
        dttp_c: S_TTP,
        ala__L_c: S_AAf['A'],
        arg__L_c: S_AAf['R'],
        asn__L_c: S_AAf['N'],
        asp__L_c: S_AAf['D'],
        cys__L_c: S_AAf['C'],
        gln__L_c: S_AAf['Q'],
        glu__L_c: S_AAf['E'],
        gly_c: S_AAf['G'],
        his__L_c: S_AAf['H'],
        ile__L_c: S_AAf['I'],
        leu__L_c: S_AAf['L'],
        lys__L_c: S_AAf['K'],
        met__L_c: S_AAf['M'],
        phe__L_c: S_AAf['F'],
        pro__L_c: S_AAf['P'],
        ser__L_c: S_AAf['S'],
        thr__L_c: S_AAf['T'],
        trp__L_c: S_AAf['W'],
        tyr__L_c: S_AAf['Y'],
        val__L_c: S_AAf['V'],
        h2o_c: S_H2O,
        adp_c: S_ADP,
        pi_c:  S_Pi,
        h_c:   S_H,
        ppi_c: S_PPi}))
    
    hostModel  = cobra.io.read_sbml_model(model)
    
    hostModel.add_reactions([plasmid_reaction])
    hostModel.reactions[-1].reversibility = False
    
    return(hostModel)

def generate_model_with_phage(model, progeny_number = 1):
    phage = "../models/genbank/m13.gbff"
    with open(phage,newline = '') as pf:
        phageFile = pf.readlines()
    phageData = str(phageFile)
    
    startG  = [jj for jj, s in enumerate(phageFile) if 'ORIGIN' in s]
    endG    = [jj for jj, s in enumerate(phageFile) if '//' in s]
    startG  = int(''.join(map(str,startG)))
    endG    = int(''.join(map(str,endG)))
    startG  = startG + 1
    
    regex           = re.compile('[^a-zA-Z]')
    phageGenome     = str(''.join(phageFile[startG:endG]))
    phageGenome     = regex.sub('',phageGenome)
    
    Cg = 1 #genome copy number per phage particle
    countA  = phageGenome.count('a')
    countC  = phageGenome.count('c')
    countG  = phageGenome.count('g')
    countT  = phageGenome.count('t')   
    antiA   = countT
    antiC   = countG
    antiG   = countC
    antiT   = countA
    
    totNTPS   = (Cg * (countA + countC + countG + countT + antiA + antiC + antiG + antiT))
    
    # mol.ntps/mol.plasmid
    V_a = (Cg*(countA + antiA))
    V_c = (Cg*(countC + antiC))
    V_g = (Cg*(countG + antiG))
    V_t = (Cg*(countT + antiT))
    # g.ntps/mol.plasmid
    G_a = V_a * ntpsDict["atp"]
    G_c = V_c * ntpsDict["ctp"]
    G_g = V_g * ntpsDict["gtp"]
    G_t = V_t * ntpsDict["ttp"]
    
    phage_genes = list(phage_dict.keys())
    phage_gene_IDs = dict.fromkeys(phage_genes)
    
    for g in range(0,len(phage_genes)):
        proteinString = '/gene="' + phage_genes[g] + '"'
        tempLine = [jj for jj, s in enumerate(phageFile) if proteinString in s]
        val = tempLine[0]
        phage_gene_IDs[phage_genes[g]] = val
    
    linesOfInterest = list(phage_gene_IDs.values())
    linesOfInterest.sort()
    Proteins = phageFile[linesOfInterest[0]-1:linesOfInterest[-1] + 16]
    
    translationLines = [jj for jj, s in enumerate(Proteins) if "/translation" in s]
    geneLines = [jj for jj, s in enumerate(Proteins) if "gene            " in s] #number of tab spaces matters!
    productLines = [jj + 1 for jj, s in enumerate(Proteins) if "gene            " in s]
    
    productLines.append(len(Proteins))
    
    translationDictionary = {}
    for i in range(0, len(productLines) - 1):
        start = productLines[i]
        end = productLines[i + 1]
        temp = Proteins[start:end]
        translationEnd = len(temp) - 1
        translationStart = [jj for jj, s in enumerate(temp) if "/translation" in s][0]
        productString = [jj for jj, s in enumerate(temp) if '/gene="' in s][0]
        geneString = temp[productString].split()[-1]
        noPunctString = geneString.translate(str.maketrans('', '', string.punctuation))
        noGeneString = noPunctString.replace("gene", "")
        translationRegion = temp[translationStart:translationEnd]
        npReg = re.compile('/translation=')
        tempgene  = str(translationRegion)
        tempgene  = npReg.sub('',tempgene)
        tempgene  = regex.sub('',tempgene)
        translationDictionary[noGeneString] = tempgene
    
    phage_proteins = list(translationDictionary.keys())
    np_list = np.zeros((20,1))
    for xx in phage_proteins:
        aaCount = np.zeros((20,1))
        protein = translationDictionary[xx]
        for ii in range(len(aaMets)):
            aaCount[ii,0] = protein.count(aaMets[ii])
        finalarray = aaCount * phage_dict[xx]
        numpy_adjusted = np.array(finalarray)
        np_list = np_list + finalarray

    total_protein_content = np_list * progeny_number
    
    # mol.aa/mol.plasmid
    V_aa    = np.zeros((20,1))
    for ii in range(len(aaMets)):
        V_aa[ii,0] = total_protein_content[ii]
    # g.a/mol.plasmid
    G_aa    = np.zeros((20,1))
    for ii in range(len(aaMets)):
        G_aa[ii,0] = V_aa[ii] * aaDict[aaMets[ii]]
        
    # Combine the final counts for nucleotide and protein costs
    M_v = (G_a + G_c + G_g + G_t) + G_aa.sum()
    
    # Nucleotides [mmol.ntps/g.virus]
    S_atp = 1000 * (V_a/M_v)
    S_ctp = 1000 * (V_c/M_v)
    S_gtp = 1000 * (V_g/M_v)
    S_ttp = 1000 * (V_t/M_v)
    
    # Amino acids [mmol.aa/g.virus]
    S_aa    = np.zeros((20,1))
    for ii in range(len(aaMets)):
        S_aa[ii] = 1000 * (V_aa[ii]/M_v)
    
    # Genome: Phosphodiester bond formation products [Pyrophosphate]
    genTemp = (((countA + countC + countG + countT) * k_ppi) - k_ppi)
    genRep  = (((antiA + antiC + antiG + antiT) * k_ppi) - k_ppi)
    genTot  = genTemp + genRep
    V_ppi   = genTot
    S_ppi   = 1000 * (V_ppi/M_v)
    
    # Protome: Peptide bond formation [ATP + H2O]
    # Note: ATP used in this process is denoated as ATPe/Ae [e = energy version]
    ppAe    = ((total_protein_content.sum() * k_atp) - k_atp)
    ppTot   = ppAe
    V_Ae    = ppTot
    S_Ae    = 1000 * (V_Ae/M_v)
    
    # Generate reaction
    
    # Left-hand terms: Nucleotides
    # Note: ATP term is a summation of genome and energy requirements
    S_ATP   = (S_atp + S_Ae) * -1
    S_CTP   = S_ctp * -1
    S_GTP   = S_gtp * -1
    S_TTP   = S_ttp * -1
    # Left-hand terms: Amino Acids
    S_AA    = S_aa * -1
    S_AAf   = dict()
    for ii in range(len(aaMets)):
        S_AAf[aaMets[ii]] = S_AA[ii,0]
    # Left-hand terms: Energy Requirements
    S_H2O   = S_Ae * -1
    # Right-hand terms: Energy Requirements
    S_ADP   = S_Ae
    S_Pi    = S_Ae
    S_H     = S_Ae
    S_PPi   = S_ppi
    
    Name = "phage"
    reaction_name       = Name + '_M13'
    phage_reaction      = Reaction(reaction_name)
    phage_reaction.lower_bound              = 0.0
    phage_reaction.upper_bound              = 1000.0
    phage_reaction.add_metabolites(({
        datp_c: S_ATP,
        dctp_c: S_CTP,
        dgtp_c: S_GTP,
        dttp_c: S_TTP,
        ala__L_c: S_AAf['A'],
        arg__L_c: S_AAf['R'],
        asn__L_c: S_AAf['N'],
        asp__L_c: S_AAf['D'],
        cys__L_c: S_AAf['C'],
        gln__L_c: S_AAf['Q'],
        glu__L_c: S_AAf['E'],
        gly_c: S_AAf['G'],
        his__L_c: S_AAf['H'],
        ile__L_c: S_AAf['I'],
        leu__L_c: S_AAf['L'],
        lys__L_c: S_AAf['K'],
        met__L_c: S_AAf['M'],
        phe__L_c: S_AAf['F'],
        pro__L_c: S_AAf['P'],
        ser__L_c: S_AAf['S'],
        thr__L_c: S_AAf['T'],
        trp__L_c: S_AAf['W'],
        tyr__L_c: S_AAf['Y'],
        val__L_c: S_AAf['V'],
        h2o_c: S_H2O,
        adp_c: S_ADP,
        pi_c:  S_Pi,
        h_c:   S_H,
        ppi_c: S_PPi}))
    
    hostModel  = cobra.io.read_sbml_model(model)
    
    hostModel.add_reactions([phage_reaction])
    hostModel.reactions[-1].reversibility = False
    return(hostModel)

def get_host_optimization(model, host_reaction):
    HVM = model
    HostRxn = host_reaction
    try:
        intTest = int(HostRxn)
        hostIdx = HostRxn
    except:
        for ii in range(len(HVM.reactions)):
            if HostRxn in str(HVM.reactions[ii]):
                hostIdx = ii
    HVM.objective = host_reaction
    hostSol     = HVM.optimize()
    hostX       = hostSol.fluxes                 # Host-optimal flux vector
    hostF       = hostSol.objective_value 
    return(hostF)

def get_plasmid_optimization(model, plasmid_reaction):
    HVM = model
    plasmidIdx    = len(HVM.reactions) - 1 
    HVM.reactions[plasmidIdx].objective_coefficient    = 0.0
    # Host optimisation
    HVM.objective = plasmid_reaction
    # Record Optima
    plasmidSol     = HVM.optimize()
    plasmidX       = plasmidSol.fluxes                 # Host-optimal flux vector
    plasmidF       = plasmidSol.objective_value 
    return(plasmidF)

def change_plasmid_bound(model, lowerbound, biomass_reaction):
    HVM = model
    plasmidIdx    = len(HVM.reactions) - 1    
    # Host optimisation
    HVM.objective = biomass_reaction
    # Record Optima
    HVM.reactions[plasmidIdx].lower_bound = lowerbound
    plasmidSol     = HVM.optimize()
    plasmidX       = plasmidSol.fluxes                 # Host-optimal flux vector
    plasmidF       = plasmidSol.objective_value 
    return(plasmidF)

def get_lowerbound_data(model, plasmid_reaction, plasmid_step_size, biomass_reaction):
    maximum_plasmid = get_plasmid_optimization(model, plasmid_reaction)
    values = {}
    for i in np.arange(0, maximum_plasmid + plasmid_step_size, plasmid_step_size):
        data = change_plasmid_bound(model, i, biomass_reaction)
        values[i] = data
        if i > 0 and list(values.values())[-1] == list(values.values())[-2]:
            print("Biomass has stopped changing")
            break
    return(values)

### Generate and validate model with phage

In [83]:
phageInfectedHost = generate_model_with_phage("../models/iJO1366.xml")

### Generate and validate model with F plasmid

In [86]:
pili_values = [0, 2, 10, 500, 1e6]
change_bound_and_pili_count = pd.DataFrame()
for i in pili_values:
    plasmidInfectedHost = generate_model_with_plasmid(i, "../models/iJO1366.xml")
    temp = get_lowerbound_data(plasmidInfectedHost, "plasmid_F", 0.1, "BIOMASS_Ec_iJO1366_core_53p95M")
    temp_df = pd.DataFrame(temp.items())
    temp_df["pili_count"] = i
    change_bound_and_pili_count = pd.concat([change_bound_and_pili_count, temp_df])

change_bound_and_pili_count = change_bound_and_pili_count.rename(columns={0: 'lower_plasmid_bound', 1: 'cell_biomass'})

In [87]:
change_bound_and_pili_count[change_bound_and_pili_count["lower_plasmid_bound"] == 0.1]

,lower_plasmid_bound,cell_biomass,pili_count
1,0.1,0.944080,0.0
1,0.1,0.871570,2.0
1,0.1,0.871383,10.0
1,0.1,0.871337,500.0
1,0.1,0.871336,1000000.0


### Export the model with the F plasmid

In [88]:
plasmidInfectedHost = generate_model_with_plasmid(10, "../models/Ecoli_F1_V3.xml")

In [89]:
write_sbml_model(plasmidInfectedHost, "../models/iJO1366_plasmid.xml")

In [90]:
phageInfectedHost = generate_model_with_phage("../models/iJO1366_plasmid.xml")

In [91]:
write_sbml_model(phageInfectedHost, "../models/iJO1366_plasmid_phage.xml")